This notebook quantizes the footprints into equal intervals of 5% based on the footprint cell weights.  Before quantizing, it upscales to the resolution of, and aligns pixels with the hyperspectral data. After Quantizing it records all intervals up to the 0.95 quantile as separate boolean masks in a stacked data array. Layer 0 corresponds to the 0.05 - 0.10 quantile contribution zone, layer 19 corresponds to 0.95 - 1, i.e. the cells in layer 19 are those that are at or above the 95th percentile of contributors to the flux at the eddy tower.   




In [ ]:
import numpy as np
import xarray as xr
import rioxarray
from xrspatial.classify import equal_interval
import os
from tqdm import tqdm 
import matplotlib.pyplot as plt
from rasterio import uint8, features
import json
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import fiona
from fiona.crs import from_epsg
import rasterio
from pandas import to_datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# paths TODO: use os.path.join to create
site = 'TEAK'
basepath = f'/media/data/NEON/{site}'
ncdf = '/media/data/NEON/TEAK/mosaic/TEAK_2019_mosaic.nc'
foot_path = '/media/data/NEON/TEAK/footprints'

test = os.path.join(basepath, 'test.tiff')

# get names of footprints
footprints = [os.path.join(foot_path, f) for f in os.listdir(foot_path) if 'summary' not in f and '.xml' not in f]

# open netcdf mosaic
data = xr.open_dataset(ncdf)

# set epsg from attrs
data = data.rio.write_crs(data.attrs['epsg'])

# clip the hyperspectral derived data to footprint extent
footprint = rioxarray.open_rasterio(footprints[0])
data = data.rio.clip_box(minx=np.atleast_1d(footprint.x.min())[0],
                        miny=np.atleast_1d(footprint.y.min())[0],
                        maxx=np.atleast_1d(footprint.x.max())[0],
                        maxy=np.atleast_1d(footprint.y.max())[0])

# ensure directory exists for vectors to live in
footmask_home = os.path.join(basepath, 'quantile_footprints')
os.makedirs(footmask_home, exist_ok=True)



for foot in tqdm(footprints[:1]):
    # get timestamp
    t = to_datetime(os.path.basename(foot).split('.')[0])

    # open footprint
    footprint = rioxarray.open_rasterio(foot)

    # footprint.shape is(1, y, x) so bust it out 3rd dim
    footprint = footprint[0]

    # rotate it because it is read in 90 off for some reason
    footprint.data = np.rot90(footprint.data, k=3)

    # then upscale
    footprint = footprint.interp_like(data.kmeans_label)

    #  transpose dims
    #footprint = footprint.transpose('x', 'y')

    # renormalize , interp messes up the sum
    footprint.data = footprint.data / np.nansum(footprint.data)

    # create equal interval contour classifcations
    footprint = equal_interval(footprint, k=20)

    # use the equal interval contours to calculate the 


In [6]:
def interp_foot_2_data(footprint, data):
   
    # footprints are different resolution so we must upscale, first get dims
    old_x = footprint.shape[1]
    old_y = footprint.shape[0]

    # then upscale
    footprint = footprint.interp_like(data.kmeans_label)

    # fill nulls
    footprint = footprint.fillna(0)

    # renormalize , interp messes up the sum
    footprint = footprint / np.nansum(footprint)

    return(footprint)


def square_crop(q):
    '''returns 1km square datarray cropped around the extent of non-zero entries'''
    # find the extent of the q95 mask
    xmin = q.x.values[np.argwhere(q.data > 0)[:,1].min()]
    xmax = q.x.values[np.argwhere(q.data > 0)[:,1].max()]

    ymin = q.y.values[np.argwhere(q.data > 0)[:,0].min()]
    ymax = q.y.values[np.argwhere(q.data > 0)[:,0].max()]

    # buffer it out to km^2
    xbuf = (1000 - (xmax - xmin)) / 2
    xmin = xmin - xbuf
    xmax = xmax + xbuf

    ybuf = (1000 - (ymax - ymin)) / 2
    ymin = ymin - ybuf
    ymax = ymax + ybuf


    mask_x = (q.x >= xmin) & (q.x <= xmax)
    mask_y = (q.y >= ymin) & (q.y <= ymax)

    crop = q.where(mask_x & mask_y, drop=True)
    return crop


def initiate_footstack(foot, data):
    '''
    uses the first footprint to create a new dataset
    '''
    # get timestamp
    t = os.path.basename(foot).split('.')[0]

    # open footprint
    footprint = rioxarray.open_rasterio(foot)

    # footprint.shape is(1, y, x) so bust it out 3rd dim
    footprint = footprint[0]

    # upscale and align footprint
    footprint = interp_foot_2_data(footprint, data)

    # create equal interval contour classifcations
    footprint = equal_interval(footprint, k=20)

    # get the q95 footprint 
    vals = set(footprint.data[footprint.data > 0].flatten())
    val_max = max(vals)
    q95 = np.zeros_like(footprint.data, dtype=np.int8)
    q95[footprint.data == val_max] = 1
    q95 = xr.DataArray(q95, dims=['y', 'x'], coords=[footprint.y, footprint.x])

    # crop q95 footprint to 1km square
    q95 = square_crop(q95)

    # align footprint to q95
    global CROP_TEMPLATE
    CROP_TEMPLATE, footprint = xr.align(q95, footprint)

    # create the dataset
    footstack = footprint.to_dataset()
    footstack = footstack.drop('equal_interval')
    
    # ad a new dimension, t, fot timeseries of footrasters 
    footstack = footstack.expand_dims(dim='t')

    # make all 0.5 t0 0.95 quantile rasters
    qs = []
    names = []

    for val in vals:
        q = np.zeros_like(footprint.data, dtype=np.int8)
        q[footprint.data == val] =  1
        q = xr.DataArray(q, dims=['y', 'x'], coords=[footprint.y, footprint.x])
        names.append(f'q{int(val * 5)}')
        qs.append(q)

    foot_series = xr.concat(qs, dim='quantile')

    return footstack, foot_series, t


def make_foot_series(foot):
    
    # get timestamp
    t = os.path.basename(foot).split('.')[0]

    # open footprint
    footprint = rioxarray.open_rasterio(foot)

    # footprint.shape is(1, y, x) so bust it out 3rd dim
    footprint = footprint[0]

    # upscale and align footprint
    footprint = interp_foot_2_data(footprint, data)

    # create equal interval contour classifcations
    footprint = equal_interval(footprint, k=20)

    # make all 0.5 t0 0.95 quantile rasters
    qs = []
    names = []

    vals = set(footprint.data[footprint.data > 0].flatten())
    for val in vals:
        q = np.zeros_like(footprint.data, dtype=np.int8)
        q[footprint.data == val] =  1
        q = xr.DataArray(q, dims=['y', 'x'], coords=[footprint.y, footprint.x])
        _ , q = xr.align(CROP_TEMPLATE, q)
        names.append(f'q{int(val * 5)}')
        qs.append(q)

    foot_series = xr.concat(qs, dim='quantile')

    return foot_series, t



first = True

for foot in tqdm(footprints):
    if first:
        footstack, foot_series, t = initiate_footstack(foot, data)
        first = False
    else:
        foot_series, t = make_foot_series(foot)
    
    foot_series = foot_series.rename('footprint') 
    foot_series.to_netcdf(os.path.join(footmask_home, f'{t}.nc'))


 26%|██▌       | 955/3672 [1:58:48<4:52:45,  6.47s/it] 